In [1]:
import rasterio
import numpy as np

# Open the raster file
with rasterio.open('CLCplus_2018_010m.tif') as src:
    # Read the raster data
    data = src.read()
    
    # Create an alpha band
    alpha = np.full(data.shape[1:], 255, dtype=rasterio.uint8)
    
    # Adjust the values in the alpha band based on the values in the raster data
    # This is just an example, adjust it to suit your needs
    alpha[data[0] < data[0].mean()] = 0
    
    # Add the alpha band to the raster data
    data_with_alpha = np.concatenate((data, alpha[None, :, :]))
    
    # Define the new raster's profile
    profile = src.profile
    profile.update(count=src.count+1, dtype=rasterio.uint8)
    
    # Write the raster data with the new alpha band to a new raster file
    with rasterio.open('output.tif', 'w', **profile) as dst:
        dst.write(data_with_alpha)

KeyboardInterrupt: 

In [19]:
src2015 = rasterio.open('dataset/trees-density/TCD_2015_020m_03035_d05.tif')
src2012 = rasterio.open('dataset/trees-density/TCD_2012_020m_03035_d03.tif')
data = src.read()

In [20]:
from rasterio.plot import show
import matplotlib.pyplot as plt
%matplotlib tk
window = rasterio.windows.Window(100*100, 50*100, 4000, 4000)

# Read a subset of the raster data
subset1 = src2012.read(window=window)
subset2 = src2015.read(window=window)

# Plot the subset of the raster data
plt.figure(figsize=(6,6))
plt.subplot(1, 2, 1)
show(subset1)
plt.title('2012')
plt.subplot(1, 2, 2)
show(subset2)
plt.show()

In [2]:
import rasterio
import numpy as np

names = [
    ("dataset/trees-density/TCD_2012_020m_03035_d03.tif","2012"),
     ("dataset/trees-density/TCD_2015_020m_03035_d05.tif","2015"),
      ("dataset/trees-density/TCD_2018_010m_03035_V2_0.tif", "2018")
]

In [2]:


# Open the raster file
with rasterio.open('dataset/trees-density/TCD_2012_020m_03035_d03.tif') as src:
    # Read the raster data
    data = src.read()

    # Create an alpha band
    alpha = np.full(data.shape[1:], 255, dtype=rasterio.uint8)

    # Adjust the values in the alpha band based on the values in the raster data
    # Pixels with higher values will be less transparent
    alpha[data[0] < data[0].mean()] = 0

    # Add the alpha band to the raster data
    data_with_alpha = np.concatenate((data, alpha[None, :, :]))

    # Define the new raster's profile
    profile = src.profile
    profile.update(count=src.count+1, dtype=rasterio.uint8)

    # Write the raster data with the new alpha band to a new raster file
    with rasterio.open('transparent_treese2012.tif', 'w', **profile) as dst:
        dst.write(data_with_alpha)

In [5]:
for name, yr in names:
    
    with rasterio.open(name) as src:
        # Read the raster data
        data = src.read()    
        # # Create an alpha band
        # alpha = np.full(data.shape[1:], 255, dtype=rasterio.uint8)
        # 
        # # Adjust the values in the alpha band based on the values in the raster data
        # # Pixels with higher values will be less transparent
        # alpha[data[0] < data[0].mean()] = 0
        # 
        # # Add the alpha band to the raster data
        # data_with_alpha = np.concatenate((data, alpha[None, :, :]))
        # 
        # # Define the new raster's profile
        # profile = src.profile
        # profile.update(count=src.count + 1, dtype=rasterio.uint8)
        # 
        # # Write the raster data with the new alpha band to a new raster file
        with rasterio.open(f"transparent_treese{yr}.tif", 'w+', **src.profile) as dst:
            print(f"Writen {yr} into transparent_treese{yr}.tif")
            dst.write(data)
        

Writen 2012 into transparent_treese2012.tif
Writen 2015 into transparent_treese2015.tif
Writen 2018 into transparent_treese2018.tif


In [6]:
import rasterio
from rasterio.enums import Resampling

fnames = ["transparent_treese2012.tif", "transparent_treese2015.tif", "transparent_treese2018.tif"]
for fname in fnames:
    with rasterio.open(fname, 'r+') as src:
        # Define the overview levels
        overviews = [2, 4, 8, 16, 32]
                                     
        # Generate overviews
        src.build_overviews(overviews, Resampling.nearest)
    
        # Update the dataset's metadata to include the overviews
        src.update_tags(ns='rio_overview', resampling='nearest')